four points to work on :-
1. rotating user agent's and other saving mechanisms
2. url join
3. is joined url is valid 
4. store set of asin ids
5. appropriate waits


In [2]:
from selenium import webdriver
from pathlib import Path
import requests
import time
import random
from itertools import repeat
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from aryaman.scripts.category.category_enums import *
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


pandas dataframe to contain main category , tag , asin id and created product links

In [3]:
products = pd.DataFrame(columns=['Main Category','Tag','ASIN','Product Link'])
asins_set = set()

functions

In [4]:
def create_url(asin_id):
    return 'https://www.amazon.in/dp/{asin_id}'.format(asin_id=asin_id)

def check_link_validity(url,session=None):
    time.sleep(random.uniform(1,7))
    if session == None:
        response = requests.get(url=url,allow_redirects=True,timeout=1000)
    else:
        response = session.get(url=url, allow_redirects=True, timeout=1000)
    # head() was giving 405 code because amazon server expects get() on those links , but get() is costly therefore
    # multithreading
    # print(response.status_code)
    try:
        if response.status_code==200 :
            if ('/dp/' in response.url or '/gp/product/' in response.url):
                return True
            else:
                print(f"Redirected to a non-product page: {response.url}")
                return False
        else:
            print("status code not 200:",response.url)
            return False
    except Exception as e:
        print(f"Exception occurred in checking link validity : {e}")
def check_asin_duplicate(set_of_asins,asin_id):
    if asin_id in set_of_asins:
        return True
    else:
        return False
    

def update_products_dataframe_and_asins_set(products_dataframe,set_of_asins,new_asin_ids_list,main_category,tag):
    urls = []
    chosen_ids = []
    for id in new_asin_ids_list:
        if not check_asin_duplicate(set_of_asins,id):
            set_of_asins.add(id)
            urls.append(create_url(id))
            chosen_ids.append(id)
    with requests.Session() as session:
        session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
        })
        with ThreadPoolExecutor(max_workers=len(urls)) as executor:
            responses = list(executor.map(check_link_validity,urls,repeat(session)))
    
    for i in range(len(responses)):
        if responses[i] == True:
            new_row = {'Main Category':main_category,'Tag':tag,'ASIN':chosen_ids[i],'Product Link':urls[i]}
            products_dataframe = pd.concat([products_dataframe,pd.DataFrame([new_row])],ignore_index=True)
            products_dataframe.reset_index(inplace=True,drop=True)
            # print(products_dataframe)
    return products_dataframe

def scrape_asin_ids(driver,css_selector_string):
    asin_elems = driver.find_elements(By.CSS_SELECTOR,css_selector_string)
    asin_ids = []
    for elem in asin_elems:
        val = elem.get_attribute("data-asin")
        if isinstance(val,str) and len(val) == 10 and val.isalnum():
            asin_ids.append(val)
    return asin_ids


global variables

In [5]:
# set them inside the code
current_main_category = None 
current_tag_of_main_category = None

driver + scraping

In [6]:
webdriver_path = "..\chromedriver.exe"
cService = webdriver.ChromeService(executable_path=webdriver_path)
chrome_options = Options()
chrome_options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=cService,options=chrome_options)
driver.maximize_window()

# Baby Products

In [7]:
current_main_category = BabyProducts.get_class_name()

## Diapers

In [8]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Diapers)

In [82]:
asin_elems_diapers = driver.find_elements(By.CSS_SELECTOR,"div[data-asin]")
asin_ids_diapers = []
for elem in asin_elems_diapers:
    val = elem.get_attribute("data-asin")
    if isinstance(val,str) and len(val) == 10 and val.isalnum():
        asin_ids_diapers.append(val)

In [181]:
len(asin_ids_diapers)

68

In [185]:
products = update_products_dataframe_and_asins_set(products,asins_set,asin_ids_diapers,current_main_category,current_tag_of_main_category)

In [187]:
products.head(5)

,Main Category,Tag,ASIN,Product Link
0,Baby Products,Diapers,B075GHP2LX,https://www.amazon.in/dp/B075GHP2LX
1,Baby Products,Diapers,B07M8HBFLK,https://www.amazon.in/dp/B07M8HBFLK
2,Baby Products,Diapers,B07SSZZRS6,https://www.amazon.in/dp/B07SSZZRS6
3,Baby Products,Diapers,B09CKXMB2T,https://www.amazon.in/dp/B09CKXMB2T
4,Baby Products,Diapers,B09M6YW5ZQ,https://www.amazon.in/dp/B09M6YW5ZQ


## Baby Wipes

In [193]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Wipes)

In [197]:
products = update_products_dataframe_and_asins_set(products,asins_set,scrape_asin_ids(driver,"div[data-asin]"),current_main_category,current_tag_of_main_category)

In [199]:
products.tail(3)

,Main Category,Tag,ASIN,Product Link
116,Baby Products,Baby Wipes,B0D9P3YJSG,https://www.amazon.in/dp/B0D9P3YJSG
117,Baby Products,Baby Wipes,B0CGXN8RWP,https://www.amazon.in/dp/B0CGXN8RWP
118,Baby Products,Baby Wipes,B0DFH93LP9,https://www.amazon.in/dp/B0DFH93LP9


## Baby Clothing

In [201]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Baby_Clothing)
products = update_products_dataframe_and_asins_set(products,asins_set,scrape_asin_ids(driver,"div[data-asin]"),current_main_category,current_tag_of_main_category)

In [202]:
products.tail(3)

,Main Category,Tag,ASIN,Product Link
186,Baby Products,Baby Clothing,B0DJRS7Y81,https://www.amazon.in/dp/B0DJRS7Y81
187,Baby Products,Baby Clothing,B099S8Q1WJ,https://www.amazon.in/dp/B099S8Q1WJ
188,Baby Products,Baby Clothing,B0CQ83HFGS,https://www.amazon.in/dp/B0CQ83HFGS


## Strollers

In [203]:
current_tag_of_main_category = BabyProducts.get_sub_name(BabyProducts.Strollers)
products = update_products_dataframe_and_asins_set(products,asins_set,scrape_asin_ids(driver,"div[data-asin]"),current_main_category,current_tag_of_main_category)

ReadTimeout: HTTPSConnectionPool(host='www.amazon.in', port=443): Read timed out. (read timeout=100)

In [1]:
products['Tag']

NameError: name 'products' is not defined